In [2]:
import glob, os
import pandas as pd
import csv
import dateutil.parser
%matplotlib inline
import matplotlib
import datetime
import calendar
import calendar
import numpy as np
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')


# Preprocessing

##Read files

In [3]:
data_path = "data/"
allFiles = [file for file in glob.glob(data_path+"*.txt")]

In [4]:
df_list=[pd.read_csv(f) for f in allFiles]
df=pd.concat(df_list)

In [5]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858


In [6]:
times_df =df.copy()
row = len(times_df.index) #counts total rows to be used as index
times_df.index = range(row)

In [7]:
times_df['ID'] = times_df.index

shifted_df = pd.DataFrame()
shifted_df['ID'] = times_df['ID']
shifted_df['SHIFTED_ENTRIES'] = times_df.ENTRIES.shift(-1)
shifted_df.fillna(0)

,ID,SHIFTED_ENTRIES
0,0,5106810
1,1,5106835
2,2,5106961
3,3,5107250
4,4,5107620
5,5,5107793
6,6,5107815
7,7,5107844
8,8,5107912
9,9,5108106


In [8]:
pd.merge(times_df, shifted_df, on='ID', how='left')

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ID,SHIFTED_ENTRIES
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635,0,5106810
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649,1,5106835
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680,2,5106961
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784,3,5107250
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858,4,5107620
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,20:00:00,REGULAR,5107620,1729914,5,5107793
6,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,00:00:00,REGULAR,5107793,1729955,6,5107815
7,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,04:00:00,REGULAR,5107815,1729964,7,5107844
8,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,08:00:00,REGULAR,5107844,1729991,8,5107912
9,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,12:00:00,REGULAR,5107912,1730068,9,5108106


## Clean columns

In [9]:
#strip column names of whitespace (for exits columns)
df.rename(columns=lambda x: x.strip(),inplace=True)

## Date parsing

In [10]:
#create 2 new columns in df 'date_time' and 'date_parsed'
df['date_time'] = pd.to_datetime(df.DATE + ' ' + df.TIME)
df['date_parsed'] = pd.to_datetime(df.DATE)

##Sort the entries

In [11]:
df.sort(['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','date_time'],inplace =True)

## Get non cummulative count

Since its Already sorted...
lets create a syntetic key by C/A, Unit,SCP,Station

In [12]:
df['key']= tuple(zip(df['C/A'],df['UNIT'],df['SCP'],df['STATION']))

Lets shift the entries,exits and count

In [13]:
df['shifted_entries']=df['ENTRIES'].shift(periods=1) 
df['shifted_exits']=df['EXITS'].shift(periods=1) 
df['shifted_key']=df['key'].shift(periods=1) 

Subtract from previous if the key is the same

In [14]:
df['ENTRIESn_hourly'] = np.where(df['shifted_key']==df['key'],df['ENTRIES']-df['shifted_entries'],0)
df['EXITSn_hourly'] = np.where(df['shifted_key']==df['key'],df['EXITS']-df['shifted_exits'],0)

The counter might have resetted to 0....lets just skip those rows

In [15]:
min_bad_value=0
max_bad_value=10000

In [16]:
bad_values=df[(df.ENTRIESn_hourly<min_bad_value) | (df.ENTRIESn_hourly>max_bad_value)]
good_values=df[(df.ENTRIESn_hourly>=min_bad_value) & (df.ENTRIESn_hourly<=max_bad_value)]
print ("There are %s bad values and %s good values"%(len(bad_values),len(good_values)))

There are 5419 bad values and 955431 good values


lets save the results and remove miscellaneous columns

In [17]:
df.drop(['shifted_entries','shifted_exits','shifted_key'],inplace=True,axis=1)
df=good_values

## Filter dates for only month of may

In [18]:
#filters date to May 2015
earliest_may_date=datetime.date(2015,5,1)
latest_may_date=datetime.date(2015,5,30)
may_df=df[(df.date_parsed >= earliest_may_date) & (df.date_parsed <= latest_may_date)]
print ("There were %s entries"%(len(df)))
print ("There are %s entries in month of may. So we lost %s entires"%(len(may_df) , len(df)-len(may_df)))

df=may_df

There were 955431 entries
There are 791432 entries in month of may. So we lost 163999 entires


## Total 

In [19]:
df=df.copy()
df['total_traffic'] = df.ENTRIESn_hourly + df.EXITSn_hourly

## Filter weekday and weekend

Moday is 0
Saturday is 5

In [20]:
#filter weekdays only
weekday_date_df=df[df['date_parsed'].dt.weekday < 5]
weekend_date_df=df[df['date_parsed'].dt.weekday >= 5]
#df[df['date_parsed'].dt.weekday < 5].tail(50)

In [21]:
wday_entries=weekday_date_df[['STATION','ENTRIESn_hourly']].groupby(['STATION']).agg({'ENTRIESn_hourly':sum})
wday_entries=wday_entries.reset_index()

wday_entries=wday_entries.sort('ENTRIESn_hourly',ascending=False)
wday_entries.rename(columns={'ENTRIESn_hourly':'entries'}, inplace=True)

wday_entries.head(20)


,STATION,entries
67,34 ST-PENN STA,3745056
72,42 ST-GRD CNTRL,3275331
66,34 ST-HERALD SQ,2580463
113,86 ST,2429077
18,14 ST-UNION SQ,2308018
74,42 ST-TIMES SQ,2112047
73,42 ST-PA BUS TE,1947096
117,96 ST,1727761
231,FULTON ST,1675386
11,125 ST,1671846


In [22]:
weekend_entries=weekend_date_df[['STATION','ENTRIESn_hourly']].groupby(['STATION']).agg({'ENTRIESn_hourly':sum})
weekend_entries=weekend_entries.reset_index()

weekend_entries=weekend_entries.sort('ENTRIESn_hourly',ascending=False)
weekend_entries.rename(columns={'ENTRIESn_hourly':'entries'}, inplace=True)

weekend_entries.head(20)

,STATION,entries
67,34 ST-PENN STA,711723
18,14 ST-UNION SQ,591556
66,34 ST-HERALD SQ,559391
113,86 ST,549964
74,42 ST-TIMES SQ,549151
73,42 ST-PA BUS TE,507479
72,42 ST-GRD CNTRL,468558
11,125 ST,417512
175,CANAL ST,388155
91,59 ST-COLUMBUS,384631


In [23]:
wday_exits=weekday_date_df[['STATION','EXITSn_hourly']].groupby(['STATION']).agg({'EXITSn_hourly':sum})
wday_exits=wday_exits.reset_index()

wday_exits=wday_exits.sort('EXITSn_hourly',ascending=False)
wday_exits.rename(columns={'EXITSn_hourly':'exits'}, inplace=True)

wday_exits.head(20)

,STATION,exits
67,34 ST-PENN STA,3194079
72,42 ST-GRD CNTRL,3031273
66,34 ST-HERALD SQ,2317637
18,14 ST-UNION SQ,2047197
113,86 ST,1870693
74,42 ST-TIMES SQ,1864992
73,42 ST-PA BUS TE,1478327
231,FULTON ST,1442471
78,47-50 ST-ROCK,1343816
11,125 ST,1330222


In [24]:
weekend_exits=weekend_date_df[['STATION','EXITSn_hourly']].groupby(['STATION']).agg({'EXITSn_hourly':sum})
weekend_exits=weekend_exits.reset_index()

weekend_exits=weekend_exits.sort('EXITSn_hourly',ascending=False)
weekend_exits.rename(columns={'EXITSn_hourly':'exits'}, inplace=True)

weekend_exits.head(20)

,STATION,exits
67,34 ST-PENN STA,649478
18,14 ST-UNION SQ,541195
66,34 ST-HERALD SQ,499624
74,42 ST-TIMES SQ,490742
113,86 ST,455201
72,42 ST-GRD CNTRL,445326
73,42 ST-PA BUS TE,415759
11,125 ST,372082
175,CANAL ST,317872
91,59 ST-COLUMBUS,294297


In [25]:
#TOTALS FIRST
#get total entries AND exits for the month per station, then get average per day

#<INSERT CODE HERE>

In [26]:
weekend_entries_exits=weekend_date_df[['STATION','ENTRIESn_hourly','EXITSn_hourly']].groupby(['STATION']).agg({'ENTRIESn_hourly':sum,'EXITSn_hourly':sum})
weekend_entries_exits=weekend_entries_exits.reset_index()

weekend_entries_exits=weekend_entries_exits.sort('ENTRIESn_hourly',ascending=False)
weekend_entries_exits.rename(columns={'ENTRIESn_hourly':'entries', 'EXITSn_hourly':'exits'}, inplace=True)

top_stations=weekend_entries_exits.head(10)
top_stations=top_stations.reset_index(drop=True)
top_stations

,STATION,entries,exits
0,34 ST-PENN STA,711723,649478
1,14 ST-UNION SQ,591556,541195
2,34 ST-HERALD SQ,559391,499624
3,86 ST,549964,455201
4,42 ST-TIMES SQ,549151,490742
5,42 ST-PA BUS TE,507479,415759
6,42 ST-GRD CNTRL,468558,445326
7,125 ST,417512,372082
8,CANAL ST,388155,317872
9,59 ST-COLUMBUS,384631,294297


In [27]:
top_station_names=top_stations['STATION'].tolist()

In [28]:
top_station_names

['34 ST-PENN STA',
 '14 ST-UNION SQ',
 '34 ST-HERALD SQ',
 '86 ST',
 '42 ST-TIMES SQ',
 '42 ST-PA BUS TE',
 '42 ST-GRD CNTRL',
 '125 ST',
 'CANAL ST',
 '59 ST-COLUMBUS']

In [29]:
top_stations.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [30]:
type(l)

NameError: name 'l' is not defined

In [ ]:
top_stations['STATION']

In [ ]:
top_stations.index.values

In [ ]:
fig=top_stations.plot(kind='bar',stacked=True);
fig.set_title('Top NY stations for the May on  Weekdays')
plt.xticks(top_stations.index.values,top_stations['STATION'])

# Top in the hours

In [ ]:
top_station_names

In [ ]:
weekday_date_df.head()

In [ ]:
top_stations

In [ ]:
weekday_date_df

In [ ]:
l=weekday_date_df.TIME.unique().tolist()
sorted(l,reverse=True)

In [ ]:
valid_time_ranges=['00:00:00', '04:00:00', '08:00:00', '12:00:00', '16:00:00', '20:00:00']

avg_wkday_entries= weekday_date_df[['STATION','DATE','TIME','ENTRIESn_hourly']].groupby(['STATION','DATE','TIME']).agg({'ENTRIESn_hourly':np.sum})
avg_wkday_entries=avg_wkday_entries.reset_index()

avg_wkday_entries = avg_wkday_entries[avg_wkday_entries['TIME'].isin(valid_time_ranges)]

avg_wkday_entries= avg_wkday_entries[['STATION','TIME','ENTRIESn_hourly']].groupby(['STATION','TIME']).agg({'ENTRIESn_hourly':np.median})
avg_wkday_entries=avg_wkday_entries.reset_index()

avg_wkday_entries = avg_wkday_entries [avg_wkday_entries['STATION'].isin(top_station_names)]

avg_wkday_entries.rename(columns={'ENTRIESn_hourly':'entries'}, inplace=True)

#wday_entries=wday_entries.sort('ENTRIESn_hourly',ascending=False)
#wday_entries.rename(columns={'ENTRIESn_hourly':'entries'}, inplace=True)
avg_wkday_entries_pivot=avg_wkday_entries.pivot(index='TIME', columns='STATION', values='entries')
avg_wkday_entries_pivot.head()



In [ ]:
avg_wkday_entries_pivot.plot()

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylabel('Entries')
plt.title('Median Station traffic over hours')

In [ ]:
#TOTALS FIRST
#plot bar graph of average count per day for May 2015, display top 10(?) busiest stations

#<INSERT CODE HERE>

In [ ]:
#BREAKDOWN BY TIME FRAME
#get total entries AND exits for the month per station grouped by timeframes "8am to 12nn", "12nn to 4pm", "4pm to 8pm"
#get average per day

#<INSERT CODE HERE>

In [ ]:
#BREAKDOWN BY TIME FRAME
#plot line graph of average count per day for May 2015, each line in graph representing 1 station

#<INSERT CODE HERE>